In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## for modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
import warnings
from sklearn.multioutput import MultiOutputRegressor

In [6]:
name= ['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height', 'Orientation', 'Glazing_Area', 'Glazing_Area_Distribution', 'Heating_Load', 'Cooling_Load']

df = pd.read_excel("data\\ENB2012_data.xlsx", names= name)



In [7]:
df.head()

### Showing top 5 columns

,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Orientation,Glazing_Area,Glazing_Area_Distribution,Heating_Load,Cooling_Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [8]:
X= df.drop(columns= ['Heating_Load', 'Cooling_Load'], axis= 1)
X.head()

,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Orientation,Glazing_Area,Glazing_Area_Distribution
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0


In [10]:
for col in df.columns:
    print(f"Categories in {col} variable: ", df[col].unique())
    print(f"-"*40)

Categories in Relative_Compactness variable:  [0.98 0.9  0.86 0.82 0.79 0.76 0.74 0.71 0.69 0.66 0.64 0.62]
----------------------------------------
Categories in Surface_Area variable:  [514.5 563.5 588.  612.5 637.  661.5 686.  710.5 735.  759.5 784.  808.5]
----------------------------------------
Categories in Wall_Area variable:  [294.  318.5 343.  416.5 245.  269.5 367.5]
----------------------------------------
Categories in Roof_Area variable:  [110.25 122.5  147.   220.5 ]
----------------------------------------
Categories in Overall_Height variable:  [7.  3.5]
----------------------------------------
Categories in Orientation variable:  [2 3 4 5]
----------------------------------------
Categories in Glazing_Area variable:  [0.   0.1  0.25 0.4 ]
----------------------------------------
Categories in Glazing_Area_Distribution variable:  [0 1 2 3 4 5]
----------------------------------------
Categories in Heating_Load variable:  [15.55  20.84  21.46  20.71  19.68  19.5   19.95

In [11]:
y = df[['Heating_Load', 'Cooling_Load']]

In [13]:
## train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape

((537, 8), (231, 8))

In [14]:
### creating an evaluate function
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [17]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": MultiOutputRegressor(Lasso()),
    "Ridge": MultiOutputRegressor(Ridge()),
    "K-Neighbors Regressor": MultiOutputRegressor(KNeighborsRegressor()),
    "Decision Tree": MultiOutputRegressor(DecisionTreeRegressor()),
    "Random Forest Regressor": MultiOutputRegressor(RandomForestRegressor()),
    "AdaBoost Regressor": MultiOutputRegressor(AdaBoostRegressor())
}

model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 3.0443
- Mean Absolute Error: 2.1343
- R2 Score: 0.9025
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 3.0896
- Mean Absolute Error: 2.2086
- R2 Score: 0.8996


Lasso
Model performance for Training set
- Root Mean Squared Error: 4.5167
- Mean Absolute Error: 3.1601
- R2 Score: 0.7869
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.6139
- Mean Absolute Error: 3.3577
- R2 Score: 0.7771


Ridge
Model performance for Training set
- Root Mean Squared Error: 3.1223
- Mean Absolute Error: 2.2475
- R2 Score: 0.8974
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 3.1623
- Mean Absolute Error: 2.3065
- R2 Score: 0.8948


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 1.6969
- Mean Absolute Error: 1.2170
- R2 Score: 0.9702
-----------------------

In [18]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
5,Random Forest Regressor,0.983912
4,Decision Tree,0.979395
3,K-Neighbors Regressor,0.946452
6,AdaBoost Regressor,0.943699
0,Linear Regression,0.899556
2,Ridge,0.894773
1,Lasso,0.777074


### Best accuraxcy given by random forest regressor algorithm